<a href="https://colab.research.google.com/github/Maheen-Sabir/colab/blob/main/forest_neuro_symbolic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!mkdir -p forest_neuro_symbolic_ai/{data/images,data/metadata,data/videos,models,outputs/images,outputs/logs}
!mkdir -p forest_neuro_symbolic_ai/src/{detection,context,rules,neuro_symbolic,utils}
!pip install ultralytics opencv-python pillow numpy
!wget https://github.com/ultralytics/assets/releases/download/v8.2.0/yolov8n.pt \
-O forest_neuro_symbolic_ai/models/yolov8n.pt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.9 MB/s eta 0:00:00
--2026-01-16 11:13:15--  https://github.com/ultralytics/assets/releases/download/v8.2.0/yolov8n.pt
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/521807533/661f1788-ea3e-404c-9bd6-57214dbb36fc?sp=r&sv=2018-11-09&sr=b&spr=https&se=2026-01-16T11%3A56%3A24Z&rscd=attachment%3B+filename%3Dyolov8n.pt&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2026-01-16T10%3A56%3A03Z&ske=2026-01-16T11%3A56%3A24Z&sks=b&skv=2018-11-09&sig=JUFjAMZonBy8FlMD8au771%2FUwQFiHXpwBf%2FwflZbcYc%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2ODU2MjI5

In [5]:
from ultralytics import YOLO

model = YOLO("forest_neuro_symbolic_ai/models/yolov8n.pt")

def detect(image_path):
    return model(image_path)

def get_time_of_day():
    return "Day"

def analyze_scene(trees):
    return "Dense Forest" if trees >= 3 else "Open Area"

def count_trees(results):
    names = results[0].names
    classes = results[0].boxes.cls

    # tree class id find karo
    tree_ids = [k for k, v in names.items() if v.lower() == "tree"]
    if not tree_ids:
        return 0

    tree_id = tree_ids[0]
    return sum(int(c.item()) == tree_id for c in classes)

def apply_rules(confidence, box_size, time_of_day, scene):
    penalty = 0
    reasons = []

    if confidence < 0.5:
        penalty += 0.2
        reasons.append("Low confidence")

    if box_size < 0.02:
        penalty += 0.25
        reasons.append("Small object size")

    if time_of_day == "Night":
        penalty += 0.1
        reasons.append("Night-time uncertainty")

    if scene == "Dense Forest":
        penalty += 0.1
        reasons.append("Dense forest occlusion")

    return penalty, reasons

def fuse(confidence, penalty, reasons):
    final_score = max(0, confidence - penalty)
    decision = "ACCEPT" if final_score >= 0.5 else "REJECT"
    return decision, final_score, reasons
image_path = "forest_neuro_symbolic_ai/data/images/lion.jpg"
results = detect(image_path)

if len(results[0].boxes) == 0:
    print("No objects detected")
    exit()

confidence = float(results[0].boxes.conf.max())

box = results[0].boxes.xywh[0]
img_h, img_w = results[0].orig_shape
box_size = (box[2] * box[3]).item() / (img_h * img_w)

trees = count_trees(results)
scene = analyze_scene(trees)
time_of_day = get_time_of_day()

penalty, reasons = apply_rules(confidence, box_size, time_of_day, scene)
decision, final_score, reasons = fuse(confidence, penalty, reasons)

print("Scene:", scene)
print("Trees detected:", trees)
print("Decision:", decision)
print("Final Score:", final_score)
print("Reasons:", reasons)



image 1/1 /content/forest_neuro_symbolic_ai/data/images/lion.jpg: 512x640 1 sheep, 204.6ms
Speed: 4.5ms preprocess, 204.6ms inference, 1.9ms postprocess per image at shape (1, 3, 512, 640)
Scene: Open Area
Trees detected: 0
Decision: ACCEPT
Final Score: 0.9334735870361328
Reasons: []
